In [1]:
from bertopic import BERTopic
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
from umap import UMAP
from hdbscan import HDBSCAN
import hdbscan
from bertopic.vectorizers import ClassTfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.representation import PartOfSpeech
import os
import json
import openml

In [2]:
datasets = openml.datasets.list_datasets()
print(openml.datasets.list_datasets()[531])

ids = list(datasets.keys())
print("Number of datasets: ", len(ids))

# Removing non-dataset entries 
ids.remove(4537)
ids.remove(4546)
ids.remove(4562)

# Fetching dataset
datasets = openml.datasets.get_datasets(ids, download_data=False, download_qualities=False)
datasets[0]

{'did': 531, 'name': 'boston', 'version': 1, 'uploader': '2', 'status': 'active', 'format': 'ARFF', 'MaxNominalAttDistinctValues': 9.0, 'NumberOfClasses': 0.0, 'NumberOfFeatures': 14.0, 'NumberOfInstances': 506.0, 'NumberOfInstancesWithMissingValues': 0.0, 'NumberOfMissingValues': 0.0, 'NumberOfNumericFeatures': 12.0, 'NumberOfSymbolicFeatures': 2.0}
Number of datasets:  5812


ExpatError: unclosed token: line 107504, column 4

In [16]:
datasets[2]

{'did': 2,
 'name': 'anneal',
 'version': 1,
 'uploader': '1',
 'status': 'active',
 'format': 'ARFF',
 'MajorityClassSize': 684.0,
 'MaxNominalAttDistinctValues': 7.0,
 'MinorityClassSize': 8.0,
 'NumberOfClasses': 5.0,
 'NumberOfFeatures': 39.0,
 'NumberOfInstances': 898.0,
 'NumberOfInstancesWithMissingValues': 898.0,
 'NumberOfMissingValues': 22175.0,
 'NumberOfNumericFeatures': 6.0,
 'NumberOfSymbolicFeatures': 33.0}

In [13]:
df = pd.DataFrame(datasets.values())
df['description'] = df['description'].fillna('')
df

KeyError: 'description'

In [3]:
for dataset in datasets:
    print(dataset)
    break

2


In [2]:
import pickle

infile = open('data.pickle','rb')
data_description_dict = pickle.load(infile)

# # Example data description
# print(data_description_dict[2])

all_data_description = []
for value in data_description_dict.values():
  all_data_description.append(value)

print(len(all_data_description))

5384


In [3]:
# Filter out None or empty strings from the descriptions
all_data_description = [desc for desc in all_data_description if desc and isinstance(desc, str)]

# Verify the data is clean
print(f"Number of descriptions: {len(all_data_description)}")


Number of descriptions: 5137


In [4]:
from bertopic import BERTopic

# Assuming `all_data_description` contains 5000 dataset descriptions
# 1. Initialize the BERTopic Model
topic_model = BERTopic(nr_topics=4, verbose=True)

# 2. Fit and Transform the Model on Descriptions
topics, probs = topic_model.fit_transform(all_data_description)

# 3. Extract Topics for Each Description
# The variable `topics` contains the topic number for each dataset description
print("Extracted Topics:")
print(topic_model.get_topic_info())

# 4. Visualize Topics (Optional)
# topic_model.visualize_topics()

# 5. Assign Topics Back to Datasets
# Create a mapping of dataset descriptions to their respective topics
dataset_topics = {f"Dataset {i}": topic for i, topic in enumerate(topics)}

# Example: Print topics for the first 10 datasets
for dataset, topic in list(dataset_topics.items())[:10]:
    print(f"{dataset}: Topic {topic}")


2024-12-04 14:28:07,198 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/161 [00:00<?, ?it/s]

2024-12-04 14:29:42,754 - BERTopic - Embedding - Completed ✓
2024-12-04 14:29:42,755 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-12-04 14:30:03,752 - BERTopic - Dimensionality - Completed ✓
2024-12-04 14:30:03,753 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-12-04 14:30:03,881 - BERTopic - Cluster - Completed ✓
2024-12-04 14:30:03,882 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-12-04 14:30:04,811 - BERTopic - Representation - Completed ✓
2024-12-04 14:30:04,811 - BERTopic - Topic reduction - Reducing number of topics
2024-12-04 14:30:05,700 - BERTopic - Topic reduction - Reduced number of topics from 147 to 4


Extracted Topics:
   Topic  Count                       Name  \
0     -1    714        -1_the_tcp_of_first   
1      0   4081            0_the_of_and_to   
2      1    281  1_tcp_first_of_disconnect   
3      2     61       2_nasa_the_of_effort   

                                      Representation  \
0  [the, tcp, of, first, disconnect, and, to, dat...   
1  [the, of, and, to, is, in, data, for, this, from]   
2  [tcp, first, of, disconnect, the, tls, alert, ...   
3  [nasa, the, of, effort, software, to, in, that...   

                                 Representative_Docs  
0  [Bleichenbacher Timing Attack: 30 micro second...  
1  [**Author**:   \n**Source**: Unknown - Date un...  
2  [Bleichenbacher Timing Attack: 2 micro seconds...  
3  [**Author**:   \n**Source**: Unknown - Date un...  
Dataset 0: Topic 0
Dataset 1: Topic 0
Dataset 2: Topic 0
Dataset 3: Topic 0
Dataset 4: Topic 0
Dataset 5: Topic 0
Dataset 6: Topic 0
Dataset 7: Topic 0
Dataset 8: Topic -1
Dataset 9: Topic 0


In [5]:
# Display Extracted Topics in Natural Language
topic_info = topic_model.get_topic_info()
print("Extracted Topics (Natural Language):")
for topic_id in topic_info['Topic']:
    if topic_id != -1:  # Exclude "outliers" (-1)
        keywords = topic_model.get_topic(topic_id)
        keywords_nl = ", ".join([word for word, _ in keywords])  # Join keywords into NL
        print(f"Topic {topic_id}: {keywords_nl}")

# Print each dataset index and its corresponding topic
print("\nDataset Index and Assigned Topics:")
for idx, topic in enumerate(topics):
    if topic != -1:  # Exclude "outliers"
        keywords = topic_model.get_topic(topic)
        keywords_nl = ", ".join([word for word, _ in keywords])  # Join keywords into NL
        print(f"Dataset {idx}: Topic {topic} - {keywords_nl}")
    else:
        print(f"Dataset {idx}: Outlier (No clear topic)")

Extracted Topics (Natural Language):
Topic 0: the, of, and, to, is, in, data, for, this, from
Topic 1: tcp, first, of, disconnect, the, tls, alert, cipher, spec, acknowledgement
Topic 2: nasa, the, of, effort, software, to, in, that, promise, and

Dataset Index and Assigned Topics:
Dataset 0: Topic 0 - the, of, and, to, is, in, data, for, this, from
Dataset 1: Topic 0 - the, of, and, to, is, in, data, for, this, from
Dataset 2: Topic 0 - the, of, and, to, is, in, data, for, this, from
Dataset 3: Topic 0 - the, of, and, to, is, in, data, for, this, from
Dataset 4: Topic 0 - the, of, and, to, is, in, data, for, this, from
Dataset 5: Topic 0 - the, of, and, to, is, in, data, for, this, from
Dataset 6: Topic 0 - the, of, and, to, is, in, data, for, this, from
Dataset 7: Topic 0 - the, of, and, to, is, in, data, for, this, from
Dataset 8: Outlier (No clear topic)
Dataset 9: Topic 0 - the, of, and, to, is, in, data, for, this, from
Dataset 10: Topic 0 - the, of, and, to, is, in, data, for, t

In [6]:
new_description = "This dataset contains information about various flowers, including petal length, width, and species classification."

# Transform the new description to predict its topic
new_topic, new_prob = topic_model.transform([new_description])

# Retrieve and print the topic keywords in NL
if new_topic[0] != -1:  # Check if it's not an outlier
    topic_keywords = topic_model.get_topic(new_topic[0])
    keywords_nl = ", ".join([word for word, _ in topic_keywords])
    print(f"Assigned Topic: {new_topic[0]} - {keywords_nl}")
else:
    print("Assigned Topic: Outlier (No clear topic)")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2024-12-04 14:30:21,260 - BERTopic - Dimensionality - Reducing dimensionality of input embeddings.
/home/taniya_das/miniconda3/envs/openml-tags/lib/python3.10/site-packages/scipy/sparse/_index.py:145: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
2024-12-04 14:30:30,970 - BERTopic - Dimensionality - Completed ✓
2024-12-04 14:30:30,971 - BERTopic - Clustering - Approximating new points with `hdbscan_model`
2024-12-04 14:30:30,972 - BERTopic - Cluster - Completed ✓


Assigned Topic: Outlier (No clear topic)


In [7]:
import pickle

# Save the model
with open("bertopic_model.pkl", "wb") as file:
    pickle.dump(topic_model, file)


Improve model

In [1]:
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic

# Load a better embedding model
embedding_model = SentenceTransformer('all-mpnet-base-v2')

# Initialize BERTopic with a custom embedding model
topic_model = BERTopic(embedding_model=embedding_model, nr_topics=4, verbose=True)
topics, probs = topic_model.fit_transform(all_data_description)



/home/taniya_das/miniconda3/envs/openml-tags/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

RuntimeError: CUDA error: CUDA-capable device(s) is/are busy or unavailable
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
topic_model = BERTopic(
    nr_topics=4, 
    min_topic_size=10,  # Minimum number of documents per topic
    n_gram_range=(1, 2),  # Use both unigrams and bigrams for topic modeling
    embedding_model=embedding_model,
    verbose=True
)

topics, probs = topic_model.fit_transform(all_data_description)


In [9]:
!pip3 freeze > requirements.txt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [16]:
import requests

url = "http://localhost:8000/predict-topic/"


description = """The dataset consists of news articles related to climate change, collected from various online sources over the past five years.
Each article includes a headline, publication date, author, article body, and tags.
The dataset covers topics such as global warming, renewable energy, carbon emissions, and environmental policies.
It is useful for analyzing trends in public discourse on climate change and for training models to classify articles into specific environmental topics."""

data = {"description": description}


response = requests.post(url, json=data)

# Print response
if response.status_code == 200:
    print("Response:", response.json())
else:
    print("Error:", response.text)


Response: {'topic_id': 0, 'keywords': 'the, of, and, to, is, in, data, for, this, from', 'probability': 0.17501574284362575}
